In [1]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [2]:
from tensorflow.keras.layers import Input, Conv1D
from tensorflow.keras import Model
import numpy as np

In [3]:
inputs = Input(shape=(20,3))
x = Conv1D(2,4,3)(inputs)
model = Model(inputs, x)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20, 3)]           0         
                                                                 
 conv1d (Conv1D)             (None, 6, 2)              26        
                                                                 
Total params: 26
Trainable params: 26
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.weights

[<tf.Variable 'conv1d/kernel:0' shape=(4, 3, 2) dtype=float32, numpy=
 array([[[-0.05840281, -0.33957317],
         [ 0.2264623 ,  0.05825561],
         [ 0.16525698,  0.23405397]],
 
        [[-0.5258953 , -0.3936214 ],
         [ 0.13454366, -0.22821242],
         [ 0.03147739, -0.01196778]],
 
        [[ 0.35090238,  0.1396572 ],
         [ 0.40993017, -0.27508488],
         [ 0.116602  , -0.148987  ]],
 
        [[ 0.37122232,  0.08273119],
         [-0.29717228,  0.53457487],
         [ 0.49348652, -0.04242468]]], dtype=float32)>,
 <tf.Variable 'conv1d/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>]

In [6]:
X = np.random.rand(1,20,3)
X

array([[[0.66937516, 0.6907002 , 0.96996117],
        [0.19520127, 0.13998107, 0.09311981],
        [0.75079076, 0.02970836, 0.07441652],
        [0.17415405, 0.83571587, 0.60334393],
        [0.10381228, 0.77579617, 0.2255741 ],
        [0.13918275, 0.12624822, 0.16357286],
        [0.69068874, 0.44852818, 0.23029813],
        [0.31139241, 0.79853943, 0.48135381],
        [0.40343184, 0.82898376, 0.89517967],
        [0.28514992, 0.60441284, 0.27842517],
        [0.77636577, 0.32528131, 0.3163831 ],
        [0.86428116, 0.89781672, 0.31025656],
        [0.50292735, 0.79692487, 0.12686924],
        [0.86170435, 0.41107323, 0.27543757],
        [0.84957032, 0.38557172, 0.99614999],
        [0.88438519, 0.51217387, 0.40840027],
        [0.4354065 , 0.75060067, 0.30749656],
        [0.18944417, 0.52536988, 0.2426369 ],
        [0.75967469, 0.98044579, 0.15048149],
        [0.80339604, 0.65717107, 0.64046179]]])

In [7]:
y = model.predict(X)
y

1/1 [==============================] - 0s 35ms/step


2023-10-23 20:02:12.254810: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


array([[[ 0.59507644,  0.45122126],
        [ 0.69210184,  0.1576352 ],
        [ 0.70753413, -0.43526217],
        [ 0.53165036, -0.09032159],
        [ 0.7328447 , -0.33714917],
        [ 0.38855883, -0.09487797]]], dtype=float32)

In [8]:
X_in = [[int(X[0][i][j]*1e36) for j in range(3)] for i in range(20)]
weights = [[[int(model.weights[0].numpy()[i][j][k]*1e36) for k in range(2)] for j in range(3)] for i in range(4)]
bias = [int(model.weights[1].numpy()[i]*1e72) for i in range(2)]
    

In [9]:
def Conv1DInt(nInputs, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    Input = [[str(input[i][j] % p) for j in range(nChannels)] for i in range(nInputs)]
    Weights = [[[str(weights[i][j][k] % p) for k in range(nFilters)] for j in range(nChannels)] for i in range(kernelSize)]
    Bias = [str(bias[i] % p) for i in range(nFilters)]
    out = [[0 for _ in range(nFilters)] for j in range((nInputs - kernelSize)//strides + 1)]
    remainder = [[None for _ in range(nFilters)] for _ in range((nInputs - kernelSize)//strides + 1)]
    for i in range((nInputs - kernelSize)//strides + 1):
        for j in range(nFilters):
            for k in range(kernelSize):
                for l in range(nChannels):
                    out[i][j] += input[i*strides + k][l]*weights[k][l][j]
            out[i][j] += bias[j]
            remainder[i][j] = str(out[i][j] % n)
            out[i][j] = str(out[i][j] // n % p)
    return Input, Weights, Bias, out, remainder

In [10]:
X_in, weights, bias, out, remainder = Conv1DInt(20, 3, 2, 4, 3, 10**36, X_in, weights, bias)
out, remainder

([['595076432565314636873373017613140151',
   '451221242188051044982898155310089176'],
  ['692101791264109605416098691984234110',
   '157635196511948127458268143134517909'],
  ['707534041076922026686504470881153950',
   '21888242871839275222246405745257275088547929138190672685567302106918640005237'],
  ['531650362682809033665109229370302980',
   '21888242871839275222246405745257275088548274078760710220327439407996310078926'],
  ['732844672853217673574394205962948992',
   '21888242871839275222246405745257275088548027251228401034660957748100577552406'],
  ['388558804761306031693830030956978129',
   '21888242871839275222246405745257275088548269522397035548207044857128138122494']],
 [['559476430118783401148220455172177920',
   '798629717274041486200430078621384704'],
  ['222725218484853070285693236889518080',
   '703236219970385258366708181735833600'],
  ['935243375755259327550534863114207232',
   '629676730817043015262169534306451456'],
  ['168613979210646022892058708579188736',
   '20078

In [11]:
in_json = {
    "in": X_in,
    "weights": weights,
    "bias": bias,
    "out": out,
    "remainder": remainder
}

In [12]:
import json

In [13]:
with open("conv1D_input.json", "w") as f:
    json.dump(in_json, f)